In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-07-02 03:14:29--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.1’

minsearch.py.1      100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-07-02 03:14:29 (22.0 MB/s) - ‘minsearch.py.1’ saved [3832/3832]



In [2]:
import minsearch

In [3]:
import json

In [4]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [5]:
# docs_raw

documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [8]:
q = 'the course has already started, can I still enroll?'

In [9]:
index.fit(documents)

In [10]:
# the higher the number, the more important
boost = { 'question': 3.0, 'section': 0.5 }

results = index.search(
    query=q,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

In [11]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [57]:
import anthropic

In [58]:
client = anthropic.Anthropic()

In [14]:
response = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": q}
    ]
)

In [15]:
response.content

[TextBlock(text="I apologize, but I don't have specific information about any particular course or institution's enrollment policies. Enrollment policies can vary widely depending on the school, program, or course in question. Here are some general points to consider:\n\n1. Many institutions have a add/drop period at the beginning of a term where late enrollment may be possible.\n\n2. Some courses might allow late enrollment with instructor permission.\n\n3. Online courses sometimes have more flexible enrollment options.\n\n4. For courses that have already progressed significantly, late enrollment might not be feasible or allowed.\n\nTo get an accurate answer, you should:\n\n1. Check the institution's website for information on late enrollment policies.\n\n2. Contact the registrar's office or admissions department directly.\n\n3. Reach out to the course instructor if possible.\n\n4. Inquire about any catch-up work that might be required if late enrollment is allowed.\n\nRemember, the s

In [16]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE.

Question: {question}

CONTEXT: {context}
""".strip()

In [17]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [18]:
print(context)

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start wit

In [19]:
prompt = prompt_template.format(question=q, context=context).strip()

In [20]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE.

Question: the course has already started, can I still enroll?

CONTEXT: section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your fi

In [21]:
response = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": prompt}
    ]
)

In [22]:
response.content

[TextBlock(text="Yes, you can still enroll in the course even though it has already started. According to the FAQ, even if you don't register, you're still eligible to submit the homeworks. However, it's important to note that there will be deadlines for turning in the final projects, so it's advised not to leave everything for the last minute.", type='text')]

In [60]:
# vid 1.5
def search(query):
    boost = { 'question': 3.0, 'section': 0.5 }

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [54]:
def build_prompt(query, search_results):
    prompt_template = """
        You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
        Use only the facts from the CONTEXT when answering the QUESTION.
        If the CONTEXT doesn't contain the answer, output NONE.

        Question: {question}

        CONTEXT: {context}
    """.strip()

    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

In [55]:
def llm(prompt):
    response = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        max_tokens=1024,
        messages=[
            {"role": "user", "content": prompt}
        ]
    )

    return response.content

In [63]:
# query = 'The course has already started, can I still enroll?'
query = 'how do i run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer

In [59]:
rag(query)

[TextBlock(text="Based on the provided context, here's how you can run Kafka:\n\n1. First, ensure that all Docker containers are up and running. You can do this by going to the directory with the docker-compose.yaml file and running:\n\n   docker compose up -d\n\n2. To run Kafka producers, consumers, or KStreams applications in Java, use the following command in the project directory:\n\n   java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n\n   Replace <jar_name> with the actual name of your JAR file.\n\n3. For Python applications:\n\n   - Create a virtual environment:\n     python -m venv env\n\n   - Activate the virtual environment:\n     On MacOS/Linux: source env/bin/activate\n     On Windows: env\\Scripts\\activate\n\n   - Install the required packages:\n     pip install -r ../requirements.txt\n\n   - Run your Python script within this virtual environment\n\n4. If you encounter permission issues with build scripts, you may need to mak

In [64]:
rag('If the course has already started, can I still enroll?')

[TextBlock(text='Based on the provided context, the answer to the question "If the course has already started, can I still enroll?" is:\n\nYes, even if you don\'t register, you\'re still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don\'t leave everything for the last minute.\n\nThe context also mentions that you can follow the course at your own pace after it finishes, as all materials will be kept available.', type='text')]

In [40]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [12]:
# Vid 1.6

In [41]:
from elasticsearch import Elasticsearch

In [42]:
es_client = Elasticsearch('http://localhost:9200')

In [43]:
# verify that we connected. Will display same info as in terminal
es_client.info()

ObjectApiResponse({'name': 'ae956fc1e85e', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'wuFnCFNmQDSpuinLtcNnzA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

In [16]:
from tqdm.auto import tqdm

In [17]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [45]:
query = 'I just discovered the course. Can I still enroll?'

In [50]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"], # ^3 indicates 3 times more important -- like boost
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [52]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer

In [59]:
rag(query)

[TextBlock(text="Based on the provided context, here's the answer to your question:\n\nYes, you can still enroll. Even if you don't register, you're still eligible to submit the homeworks. However, be aware that there will be deadlines for turning in the final projects, so don't leave everything for the last minute.\n\nIt's worth noting that registration is mainly used to gauge interest before the start date, and it's not checked against any registered list when submitting homework. You can start learning and submitting homework without formally registering.", type='text')]